## 誤差逆伝播とレイヤーの実装

In [17]:
import numpy as np

### 乗算レイヤー

$z = x \times y$ のとき、 $\frac{\partial z}{\partial x} = y$, $\frac{\partial z}{\partial y} = x$ であるから、乗算を行うノードは $x$ と $y$ の値を保持しておく必要がある。

In [18]:
# 乗算を行うレイヤー
class MulLayer:

    def __init__(self):
        self.x = None
        self.y = None

    def forward(self, x, y):
        # x, yの値を保持
        self.x = x
        self.y = y
        # 計算結果を返す
        return x * y

    def backward(self, dout):
        # 保持していたx, yの値と、下流から逆伝播してきた微分値を掛け合わせて返す
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy

In [19]:
# リンゴの問題を実装
apple = 100
apple_num = 2
tax = 1.1

# レイヤー
mulAppleLayer = MulLayer()
mulTaxLayer = MulLayer()

# 順伝播（金額計算）
apple_price = mulAppleLayer.forward(apple, apple_num)
price = mulTaxLayer.forward(apple_price, tax)
print(price)

# 逆伝播
dprice = 1
dapple_price, dtax = mulTaxLayer.backward(dprice)
dapple, dapple_num = mulAppleLayer.backward(dapple_price)
print(dapple, dapple_num, dtax)

220.00000000000003
2.2 110.00000000000001 200


### 加算レイヤー

微分値は $1$ なので、何も特別な操作をする必要はない

In [20]:
# 加算を行うレイヤー
class AddLayer:

    def __init__(self):
        # 初期化処理は特に必要ない
        pass

    def forward(self, x, y):
        return x + y

    def backword(self, dout):
        # 入力された微分値をそのまま返す
        return dout, dout

### リンゴとみかんの問題を実装

In [21]:
apple = 100
appleNum = 2
orange = 150
orangeNum = 3
tax = 1.1

# レイヤーを定義
mulAppleLayer = MulLayer()
mulOrangeLayer = MulLayer()
addAppleOrangeLayer = AddLayer()
mulTaxLayer = MulLayer()

# 順伝播
applePrice = mulAppleLayer.forward(apple, appleNum)
orangePrice = mulOrangeLayer.forward(orange, orangeNum)
subtotal = addAppleOrangeLayer.forward(applePrice, orangePrice)
total = mulTaxLayer.forward(subtotal, tax)
print(f"合計金額：{total}")

# 逆伝播
dprice = 1  # 初期の微分値
dSubtotal, dTax = mulTaxLayer.backward(dprice)  # 小計、税率
dApplePrice, dOrangePrice = addAppleOrangeLayer.backword(
    dSubtotal
)  # リンゴ合計、みかん合計
dOrange, dOrangeNum = mulOrangeLayer.backward(
    dOrangePrice
)  # みかんの単価、みかんの個数
dApple, dAppleNum = mulAppleLayer.backward(dApplePrice)  # みかんの単価、みかんの個数
print(dAppleNum, dApple, dOrange, dOrangeNum, dTax)

合計金額：715.0000000000001
110.00000000000001 2.2 3.3000000000000003 165.0 650


### ReLUレイヤー

In [ ]:
# ReLUレイヤー
class ReLU:

    def __init__(self):
        self.mask = None

    def forward(self, x: np.ndarray):
        self.mask = x <= 0  # 行列の要素が0以下の場所を記録
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout: np.ndarray):
        dout[self.mask] = 0
        return dout

### Sigmoidレイヤー

In [ ]:
# Sigmoidレイヤー
class Sigmoid:

    def __init__(self) -> None:
        self.out = None

    def forward(self, x: np.ndarray):
        # ブロードキャストで、要素ごとに計算
        self.out = 1 / (1 + np.exp(-x))
        return self.out

    def backward(self, dout: np.ndarray):
        return dout * (1.0 - self.out) * self.out  # type: ignore